# Semantic Search

The Semantic search API helps you build AI systems that use data from documents. With the API, you can search for texts with similar semantic meaning, unlike the search where you search for keywords, and find relevant texts written in other languages.

The Semantic search API calculates Vector Embeddings for all PDF documents uploaded to Cognite Data Fusion (CDF). You can read more about [embeddings](https://platform.openai.com/docs/guides/embeddings) and [AIs multitool vector embeddings](https://cloud.google.com/blog/topics/developers-practitioners/meet-ais-multitool-vector-embeddings).

Vector embeddings are created as part of the built-in Retrieval Augmented Generation (RAG) pipeline. All PDF documents you upload to CDF are parsed and OCRed, and the extracted text is divided into suitably sized passages which are indexed into a vector store.

This notebook shows you how to upload a PDF to CDF and use the Semantic search API to find relevant passages for a user question.

In [ ]:
from cognite.client import CogniteClient

# Instantiate Cognite SDK client:
client = CogniteClient()

## Step 1. Upload PDF

You can upload a PDF file to CDF one of the following ways:

* Go to **_CDF_** > **_Industrial tools_** > **_Canvas_** and drag your PDF file to the canvas or upload existing files by selecting **_+ Add data_**.  
  If you don't have a good file to upload, try this [test file](./well_report.pdf).

* Go to **_CDF_** > **_Industrial tools_** > **_Data explorer_** > **_Files_** and select **_Upload_**.

* Use the Python code.

In [ ]:
response1 = client.files.upload(path="./well_report.pdf")
document_id = response1.id
print(document_id)

## Step 2. Processing

Once you've uploaded the file, wait for it to pass through the RAG pipeline. You can use the Document status API to poll the status.

In [ ]:
import time

status_path = f"/api/v1/projects/{client.config.project}/documents/status"

body = {
    "items": [
        {
            "id": document_id
        }
    ]
}

while True:
    response2 = client.post(status_path, json=body, headers={"cdf-version": "beta"}).json()

    status = response2["items"][0]["passages"]["status"]
    print(f"status: {status}")

    if status in {"waiting", "running"}:
        time.sleep(5)
        continue

    break

## Step 3. Search

Once the document is fully indexed, search for the the relevant pasages with the Python code.

In [ ]:
import json

search_path = f"/api/v1/projects/{client.config.project}/documents/passages/search"

body = {
    "limit": 3,
    "filter": {
        "and": [
            {
                "equals": {
                    "property": ["document", "id"],
                    "value": document_id
                }
            },
            {
                "semanticSearch": {
                    "property": ["content"],
                    "value": "Where is the Volve field located?"
                }
            }
        ]
    }
}

response3 = client.post(search_path, json=body).json()

print(json.dumps(response3, indent=2))